In [1]:
import utils as utils
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
DATA_FILE_PATH = 'data.json'

In [3]:
train_tups, test_tups = utils.split_data(utils.generate_tuples_from_file(DATA_FILE_PATH, num_samples=10000), test_size=0.2)

print('Training set size: ', len(train_tups[0]))
print('Test set size: ', len(test_tups[0]))
print('Sample tokenized review: ', train_tups[0][0])
print('Sample rating: ', train_tups[1][0])

Training set size:  8000
Test set size:  2000
Sample tokenized review:  ['I', 'really', 'ca', "n't", 'think', 'of', 'anything', 'bad', 'to', 'say', 'about', 'this', 'place', '!', 'I', 'have', 'eaten', 'there', 'twice', 'and', 'verbally', '(', 'and', 'loudly', ')', 'moaned', 'in', 'food', 'pleasure', 'over', 'each', 'meal', '!', 'Meal', 'One', ':', 'We', 'Appz', "'d", 'on', 'the', 'Assorted', 'Breads', '.', 'Safe', 'choice', ',', 'but', 'wonderful', 'selections', '!', 'My', 'dinner', 'entree', 'was', 'the', 'Amish', 'Chicken', '.', 'Very', 'tender', 'and', 'juicy', 'and', 'the', 'potato', 'hash', 'was', 'wonderful', '!', 'Meal', 'Two', ':', 'The', 'Appz', 'was', 'the', 'Potted', 'Pig', ',', 'and', 'the', 'Flash', 'Fried', 'Brussels', 'Sprouts', '.', 'The', 'potted', 'pig', 'was', 'plated', 'creatively', 'having', 'to', 'use', 'a', 'spoon', 'to', 'dive', 'into', 'the', 'pork', '(', 'texture', 'of', 'a', 'country', 'pate', ')', 'and', 'then', 'add', 'your', 'toppings', 'from', 'the', 'pic

In [4]:
# implement FFNN with doc2vec embeddings as input

tagged_docs = [TaggedDocument(doc, [i]) for i, doc in enumerate(train_tups[0])]
d2v_model = Doc2Vec(tagged_docs, vector_size=100, window=2, min_count=1, workers=4)

vectors = [d2v_model.dv[i] for i in range(len(tagged_docs))]

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.metrics import Recall, Precision, F1Score

input_dim = 100

model = Sequential()
model.add(Dense(100, activation='relu', input_shape=(input_dim,)))  
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))


# put in an output layer
model.add(Dense(5, activation='softmax'))

model.summary()
sgd = SGD(learning_rate=.01)
recall = Recall()
precision = Precision()
f1_score = F1Score()

# call compile here
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', recall, precision, f1_score])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               10100     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                 255       
                                                                 
Total params: 15405 (60.18 KB)
Trainable params: 15405 (60.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
import tensorflow as tf
import numpy as np

# Convert input data to NumPy arrays
vectors_np = np.array(vectors)
train_tups_np = train_tups[1]
train_tups_np = utils.get_one_hot_encodings(train_tups_np)

# Create a tf.data.Dataset from the input and output vectors
dataset = tf.data.Dataset.from_tensor_slices((vectors_np, train_tups_np))

# Repeat the dataset indefinitely
dataset = dataset.repeat()

# Batch the data
batch_size = 5000
dataset = dataset.batch(batch_size)

# Fit the model
steps_per_epoch = len(vectors_np) // batch_size
model.fit(dataset, epochs=20, steps_per_epoch=steps_per_epoch)

Epoch 1/20
1/1 [==============================] - 0s 22ms/step - loss: 0.8412 - accuracy: 0.6444 - recall: 0.5028 - precision: 0.7429 - f1_score: 0.5436
Epoch 2/20
1/1 [==============================] - 0s 10ms/step - loss: 0.8439 - accuracy: 0.6416 - recall: 0.5118 - precision: 0.7322 - f1_score: 0.5489
Epoch 3/20
1/1 [==============================] - 0s 10ms/step - loss: 0.8385 - accuracy: 0.6462 - recall: 0.5106 - precision: 0.7443 - f1_score: 0.5508
Epoch 4/20
1/1 [==============================] - 0s 10ms/step - loss: 0.8445 - accuracy: 0.6468 - recall: 0.5140 - precision: 0.7436 - f1_score: 0.5527
Epoch 5/20
1/1 [==============================] - 0s 9ms/step - loss: 0.8469 - accuracy: 0.6428 - recall: 0.5092 - precision: 0.7299 - f1_score: 0.5550
Epoch 6/20
1/1 [==============================] - 0s 11ms/step - loss: 0.8258 - accuracy: 0.6562 - recall: 0.5176 - precision: 0.7521 - f1_score: 0.5596
Epoch 7/20
1/1 [==============================] - 0s 12ms/step - loss: 0.8380 - acc

In [28]:
review_to_predict = "The food was awful."
vectors_np = np.array([d2v_model.infer_vector(review_to_predict.split(' '))])
prob_distribution = model.predict(vectors_np)

print('Predicted rating: ', np.argmax(prob_distribution) + 1)



1/1 [==============================] - 0s 15ms/step
Predicted rating:  2


In [32]:
review_to_predict = "The waiter was nice. The food was warm and delicious. I would definitely come back again."
vectors_np = np.array([d2v_model.infer_vector(review_to_predict.split(' '))])
prob_distribution = model.predict(vectors_np)

print('Predicted rating: ', np.argmax(prob_distribution) + 1)



1/1 [==============================] - 0s 14ms/step
Predicted rating:  4
